<a href="https://colab.research.google.com/github/rupakdude24/janatahack-computer-vision-hackathon/blob/master/Computer_Vision_Inception_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyDrive

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import resize
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '1Va3e0gOjYXZI2hKUkTKCW1_cp3DxjXPi'})
download.GetContentFile('train_SOaYf6m.zip')
!unzip train_SOaYf6m.zip

Archive:  train_SOaYf6m.zip
   creating: images/
  inflating: images/0.jpg            
  inflating: images/1.jpg            
  inflating: images/10.jpg           
  inflating: images/100.jpg          
  inflating: images/1000.jpg         
  inflating: images/1001.jpg         
  inflating: images/1002.jpg         
  inflating: images/1003.jpg         
  inflating: images/1004.jpg         
  inflating: images/1005.jpg         
  inflating: images/1006.jpg         
  inflating: images/1007.jpg         
  inflating: images/1008.jpg         
  inflating: images/1009.jpg         
  inflating: images/101.jpg          
  inflating: images/1010.jpg         
  inflating: images/1011.jpg         
  inflating: images/1012.jpg         
  inflating: images/1013.jpg         
  inflating: images/1014.jpg         
  inflating: images/1015.jpg         
  inflating: images/1016.jpg         
  inflating: images/1017.jpg         
  inflating: images/1018.jpg         
  inflating: images/1019.jpg         
 

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-06-17 13:51:04--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

inception_v3_weight 100%[===================>]  83.84M   321MB/s    in 0.3s    

2020-06-17 13:51:04 (321 MB/s) - ‘inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
weights1="inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
pre_trained_model = InceptionV3(input_shape = (224, 224, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(weights1)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


In [ ]:
from tensorflow.keras.optimizers import Adam
x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = Adam(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
train_image = []
for img_name in tqdm(train['image_names']):
    # defining the image path
    image_path = 'images/' + img_name
    # reading the image
    img = imread(image_path)
    # normalizing the pixel values
    img = img/255
    # resizing the image to (224,224,3)
    img = resize(img, output_shape=(224,224,3), mode='constant', anti_aliasing=True)
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    train_image.append(img)

# converting the list to numpy array
train_x = np.array(train_image)
train_x.shape

100%|██████████| 1646/1646 [00:17<00:00, 95.12it/s]


(1646, 224, 224, 3)

In [ ]:
train_y = train['emergency_or_not'].values
x_train, x_val, y_train, y_val = train_test_split(train_x, train_y, test_size = 0.1, random_state = 13, stratify=train_y)
(x_train.shape, y_train.shape), (x_val.shape, y_val.shape)

(((1481, 224, 224, 3), (1481,)), ((165, 224, 224, 3), (165,)))

In [ ]:
train_datagen.fit(x_train)
test_datagen.fit(y_train)

In [ ]:
from keras.callbacks import ModelCheckpoint
batch_size = 32
epochs = 30

history = model.fit(
    x_train, y_train, batch_size=batch_size,
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val))

Epoch 1/30
46/46 [==============================] - 4s 88ms/step - loss: 0.3824 - accuracy: 0.8370 - val_loss: 0.1923 - val_accuracy: 0.9212
Epoch 2/30
46/46 [==============================] - 4s 87ms/step - loss: 0.0732 - accuracy: 0.9738 - val_loss: 0.1344 - val_accuracy: 0.9455
Epoch 3/30
46/46 [==============================] - 4s 86ms/step - loss: 0.0183 - accuracy: 0.9952 - val_loss: 0.1466 - val_accuracy: 0.9333
Epoch 4/30
46/46 [==============================] - 4s 87ms/step - loss: 0.0107 - accuracy: 0.9965 - val_loss: 0.1238 - val_accuracy: 0.9515
Epoch 5/30
46/46 [==============================] - 4s 87ms/step - loss: 0.0101 - accuracy: 0.9965 - val_loss: 0.1334 - val_accuracy: 0.9455
Epoch 6/30
46/46 [==============================] - 4s 87ms/step - loss: 0.0082 - accuracy: 0.9959 - val_loss: 0.1445 - val_accuracy: 0.9455
Epoch 7/30
46/46 [==============================] - 4s 87ms/step - loss: 0.0074 - accuracy: 0.9979 - val_loss: 0.1356 - val_accuracy: 0.9515
Epoch 8/30
46

In [ ]:
score = model.evaluate(x_val, y_val)

print('Loss: {:.4f}'.format(score[0]))
print('Accuracy: {:.4f}'.format(score[1]))

6/6 [==============================] - 0s 53ms/step - loss: 0.1377 - accuracy: 0.9576
Loss: 0.1377
Accuracy: 0.9576


In [ ]:
download = drive.CreateFile({'id': '1GbxkIK2lvFng4DY9lba6MA26HE1jp6XY'})
download.GetContentFile('test_vc2kHdQ.csv')
test= pd.read_csv('test_vc2kHdQ.csv')

In [ ]:
test_image = []
for img_name in tqdm(test['image_names']):
    # defining the image path
    image_path = 'images/' + img_name
    # reading the image
    img = imread(image_path)
    # normalizing the pixel values
    img = img/255
    # resizing the image to (224,224,3)
    img = resize(img, output_shape=(224,224,3), mode='constant', anti_aliasing=True)
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    test_image.append(img)

# converting the list to numpy array
test = np.array(test_image)
test.shape

100%|██████████| 706/706 [00:07<00:00, 91.56it/s]


(706, 224, 224, 3)

In [ ]:
prediction = model.predict(test)

In [ ]:
prediction[100]

array([0.99985445], dtype=float32)

In [ ]:
y_classes = np.where(prediction > 0.6, 1,0)

In [ ]:
download = drive.CreateFile({'id': '1HoSOyIERKK_1xF0XEML-kZWkYVqZweNZ'})
download.GetContentFile('sample_submission_yxjOnvz.csv')

In [ ]:
sample = pd.read_csv('sample_submission_yxjOnvz.csv')

sample['emergency_or_not'] = y_classes
sample.to_csv('sample1.csv', header=True, index=False)